In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_TRACING_V2 = os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_ENDPOINT = os.getenv("LANGCHAIN_ENDPOINT")
LANGCHAIN_PROJECT_ID = os.getenv("LANGCHAIN_PROJECT_ID")

지원되는 모델 목록

<table>

<thead>
<tr>
<th>MODEL</th>
<th>PAGES PER DOLLAR</th>
<th>PERFORMANCE ON MTEB EVAL</th>
<th>MAX INPUT</th>
</tr>
</thead>
<tbody>
<tr>
<td>text-embedding-3-small</td>
<td>62,500</td>
<td>62.3%</td>
<td>8191</td>
</tr>
<tr>
<td>text-embedding-3-large</td>
<td>9,615</td>
<td>64.6%</td>
<td>8191</td>
</tr>
<tr>
<td>text-embedding-ada-002</td>
<td>12,500</td>
<td>61.0%</td>
<td>8191</td>
</tr>
</tbody>
</table>

In [2]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model = 'text-embedding-3-small')

In [3]:
text = "임베딩 테스트를 위한 샘플 문장입니다."

## 1. Quary Embedding

In [4]:
quary_result=embeddings.embed_query(text)

In [6]:
len(quary_result)

1536

## 2. Document embeddings

In [9]:
doc_result = embeddings.embed_documents(
    [text, text, text, text])

In [10]:
doc_result[0][:5]

[-0.012475714087486267,
 0.04283829405903816,
 0.016534078866243362,
 -0.025209102779626846,
 0.0133238909766078]